In [15]:
using DrWatson

In [16]:
@quickactivate "RxInferThesisExperiments"

In [17]:
using RxInferThesisExperiments, Turing, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, ProgressMeter, Logging

In [18]:
const bfolder_nuts = datadir("hgf", "turing", "nuts")

"/Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/turing/nuts"

In [19]:
# Pregenerate paths for benchmark data
mkpath(bfolder_nuts);

In [20]:
const environment = HGFEnvironment()

HGFEnvironment()

In [21]:
# Include the model specification
include(srcdir("models", "turing", "hgf.jl"));

[ Info: [Turing]: progress logging is disabled globally
[ Info: [AdvancedVI]: global PROGRESS is set as false


In [22]:
function run_benchmark_nuts(params)
    return with_logger(NullLogger()) do
        @unpack T, nsamples, seed = params

        zstates, xstates, observations = rand(StableRNG(seed), environment, T);
        model    = (observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior) -> begin 
            HGF(observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior, environment.kappa, environment.omega)
        end
        result   = run_inference(model, observations; nsamples = nsamples, rng = StableRNG(seed))
        e_states = extract_posteriors(T, result)
        z_amse   = compute_amse(zstates, e_states[:z])
        x_amse   = compute_amse(xstates, e_states[:x])

        # Turing need to recreate the model every time....
        benchmark_modelcreation = @benchmark begin end

        benchmark_inference = @benchmark run_inference(model, observations; nsamples = $nsamples, rng = StableRNG($seed)) setup=begin
            model = (observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior) -> begin 
                HGF(observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior, environment.kappa, environment.omega)
            end
            zstates, xstates, observations = rand(StableRNG($seed), environment, $T);
        end

        emse = compute_emse(seed) do _seed
            local zstates, xstates, observations = rand(StableRNG(_seed), environment, T);
            local model = (observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior) -> begin 
                HGF(observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior, environment.kappa, environment.omega)
            end
            local result   = run_inference(model, observations; nsamples = nsamples, rng = StableRNG(_seed))
            local e_states = extract_posteriors(T, result)
            return [ compute_amse(zstates, e_states[:z]), compute_amse(xstates, e_states[:x]) ]
        end

        z_emse = emse[1]
        x_emse = emse[2]

        states = (z = zstates, x = xstates)

        output = @strdict T nsamples seed states e_states observations z_amse x_amse z_emse x_emse benchmark_modelcreation benchmark_inference

        return output
    end
end

run_benchmark_nuts (generic function with 1 method)

In [23]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params_nuts = dict_list(Dict(
    "T"           => [ 10, 20, 30, 100, 300 ],
    "nsamples"    => [ 100, 200 ],
    "seed"        => [ 42 ]
));

In [24]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks_nuts = map(benchmark_params_nuts) do params
    result, _ = produce_or_load(run_benchmark_nuts, bfolder_nuts, params; tag = false, force = false)
    return result
end;

[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/turing/nuts/T=10_nsamples=100_seed=42.jld2 does not exist. Producing it now...
Progress: 100%|█████████████████████████████████████████| Time: 0:00:14
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|████████████████████████████████████████

Progress: 100%|█████████████████████████████████████████| Time: 0:02:27
Progress: 100%|█████████████████████████████████████████| Time: 0:02:22
Progress: 100%|█████████████████████████████████████████| Time: 0:02:08
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/turing/nuts/T=300_nsamples=100_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/turing/nuts/T=10_nsamples=200_seed=42.jld2 does not exist. Producing it now...
Progress: 100%|█████████████████████████████████████████| Time: 0:00:11
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10
Pr

Progress: 100%|█████████████████████████████████████████| Time: 0:05:15
Progress: 100%|█████████████████████████████████████████| Time: 0:05:15
Progress: 100%|█████████████████████████████████████████| Time: 0:05:22
Progress: 100%|█████████████████████████████████████████| Time: 0:05:36
Progress: 100%|█████████████████████████████████████████| Time: 0:04:52
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/turing/nuts/T=300_nsamples=200_seed=42.jld2 saved.


In [28]:
sort(prepare_benchmarks_table(bfolder_nuts), [ :T ])

[ Info: Scanning folder /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/turing/nuts for result files.
[ Info: Added 10 entries.


Row,T,nsamples,z_emse,x_emse,z_amse,seed,x_amse,inference,creation
,Int64?,Int64?,Float64?,Float64?,Float64?,Int64?,Float64?,Tuple…?,Tuple…?
1,10,100,0.443523,3.23022,0.0691279,42,3.22987,"(4.28324e9, 4.29914e9, 4.16533e8)","(0.75, 0.885107, 0.0)"
2,10,200,1.08465,3.71429,0.548014,42,3.65631,"(1.00265e10, 1.00265e10, 9.75395e8)","(0.791, 0.882136, 0.0)"
3,20,100,0.373154,5.5134,0.0762052,42,10.183,"(8.96278e9, 8.96278e9, 8.63574e8)","(0.791, 0.885749, 0.0)"
4,20,200,0.796753,5.38577,0.294512,42,10.4223,"(2.03313e10, 2.03313e10, 2.04434e9)","(0.791, 0.881699, 0.0)"
5,30,100,0.403985,10.6999,0.19904,42,18.018,"(1.36251e10, 1.36251e10, 1.3427e9)","(0.75, 0.882046, 0.0)"
6,30,200,0.64423,10.5016,0.206156,42,16.1603,"(3.10687e10, 3.10687e10, 3.23906e9)","(0.791, 0.879969, 0.0)"
7,100,100,0.357281,40.4529,0.276982,42,140.964,"(4.6841e10, 4.6841e10, 5.09458e9)","(0.792, 0.884274, 0.0)"
8,100,200,0.403341,40.9612,0.300429,42,147.885,"(1.00988e11, 1.00988e11, 1.18473e10)","(0.791, 0.883692, 0.0)"
9,300,100,0.360795,190.802,0.452446,42,466.247,"(1.37325e11, 1.37325e11, 1.69889e10)","(0.791, 0.874617, 0.0)"


# Versions

In [26]:
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 10 × Apple M2 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 2 on 6 virtual cores


In [27]:
] status

Project RxInferThesisExperiments v1.0.0
Status `~/.julia/dev/RxInferThesisExperiments/Project.toml`
  [b5ca4192] AdvancedVI v0.2.3
  [6e4b80f9] BenchmarkTools v1.3.2
  [76274a88] Bijectors v0.12.5
  [35d6a980] ColorSchemes v3.21.0
  [a93c6f00] DataFrames v1.5.0
  [31c24e10] Distributions v0.25.95
  [634d3b9d] DrWatson v2.12.5
  [442a2c76] FastGaussQuadrature v0.5.1
  [9fc3f58a] ForneyLab v0.12.0
  [f6369f11] ForwardDiff v0.10.35
  [14197337] GenericLinearAlgebra v0.3.11
  [19dc6840] HCubature v1.5.1
  [7073ff75] IJulia v1.24.2
  [b964fa9f] LaTeXStrings v1.3.0
  [bdcacae8] LoopVectorization v0.12.159
  [3bd65402] Optimisers v0.2.18
  [8314cec4] PGFPlotsX v1.6.0
  [e4faabce] PProf v2.2.2
  [91a5bcdd] Plots v1.38.15
  [92933f4c] ProgressMeter v1.7.2
  [37e2e3b7] ReverseDiff v1.14.6
  [86711068] RxInfer v2.11.1
  [860ef19b] StableRNGs v1.0.0
  [aedffcd0] Static v0.8.7
  [90137ffa] StaticArrays v1.5.25
⌅ [2913bbd2] StatsBase v0.33.21
  [4c63d2b9] StatsFuns v1.3.0
  [fce5fe82] Turing v0.25.1